<a href="https://colab.research.google.com/github/gouravdhalwal08/Cricket-Match-Outcome-Predictor-using-Exact-Inference/blob/main/Cricket_Match_Outcome_Predictor_using_Exact_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pgmpy

In [ ]:
# Import necessary libraries
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
# Step 1: Define the structure of the Bayesian Network
# Nodes: Team Strength, Weather, Pitch Condition, Outcome
model = BayesianNetwork([('Team Strength', 'Outcome'),
                         ('Weather', 'Outcome'),
                         ('Pitch Condition', 'Outcome')])

In [ ]:

# Step 2: Define the Conditional Probability Distributions (CPDs)

# CPD for Team Strength (1: Strong, 0: Weak)
cpd_team_strength = TabularCPD(variable='Team Strength', variable_card=2,
                               values=[[0.7], [0.3]])  # 70% Strong, 30% Weak

In [ ]:
print(cpd_team_strength)

+------------------+-----+
| Team Strength(0) | 0.7 |
+------------------+-----+
| Team Strength(1) | 0.3 |
+------------------+-----+


In [ ]:
# CPD for Weather (1: Good, 0: Bad)
cpd_weather = TabularCPD(variable='Weather', variable_card=2,
                         values=[[0.8], [0.2]])  # 80% Good, 20% Bad

In [ ]:
print(cpd_weather)

+------------+-----+
| Weather(0) | 0.8 |
+------------+-----+
| Weather(1) | 0.2 |
+------------+-----+


In [ ]:
# CPD for Pitch Condition (1: Favorable, 0: Unfavorable)
cpd_pitch_condition = TabularCPD(variable='Pitch Condition', variable_card=2,
                                 values=[[0.6], [0.4]])  # 60% Favorable, 40% Unfavorable


In [ ]:
print(cpd_pitch_condition)

+--------------------+-----+
| Pitch Condition(0) | 0.6 |
+--------------------+-----+
| Pitch Condition(1) | 0.4 |
+--------------------+-----+


In [ ]:
# CPD for Outcome (1: Win, 0: Loss)
cpd_outcome = TabularCPD(variable='Outcome', variable_card=2,
                         values=[[0.9, 0.7, 0.6, 0.4, 0.3, 0.1, 0.5, 0.2],  # Prob of Win
                                 [0.1, 0.3, 0.4, 0.6, 0.7, 0.9, 0.5, 0.8]],  # Prob of Loss
                         evidence=['Team Strength', 'Weather', 'Pitch Condition'],
                         evidence_card=[2, 2, 2])

In [ ]:
print(cpd_outcome)

+-----------------+--------------------+-----+--------------------+--------------------+
| Team Strength   | Team Strength(0)   | ... | Team Strength(1)   | Team Strength(1)   |
+-----------------+--------------------+-----+--------------------+--------------------+
| Weather         | Weather(0)         | ... | Weather(1)         | Weather(1)         |
+-----------------+--------------------+-----+--------------------+--------------------+
| Pitch Condition | Pitch Condition(0) | ... | Pitch Condition(0) | Pitch Condition(1) |
+-----------------+--------------------+-----+--------------------+--------------------+
| Outcome(0)      | 0.9                | ... | 0.5                | 0.2                |
+-----------------+--------------------+-----+--------------------+--------------------+
| Outcome(1)      | 0.1                | ... | 0.5                | 0.8                |
+-----------------+--------------------+-----+--------------------+--------------------+


In [ ]:
# Step 3: Add CPDs to the model
model.add_cpds(cpd_team_strength, cpd_weather, cpd_pitch_condition, cpd_outcome)


In [ ]:

# Step 4: Validate the model
assert model.check_model()

In [ ]:
# Step 5: Perform Exact Inference using Variable Elimination
inference = VariableElimination(model)

In [ ]:
# Example Queries:
# Query 1: Given that the team is strong, weather is good, and the pitch is favorable,
# What is the probability of winning?
result = inference.query(variables=['Outcome'], evidence={'Team Strength': 1, 'Weather': 1, 'Pitch Condition': 1})
print("\nPrediction with Strong Team, Good Weather, Favorable Pitch:")
print(result)


Prediction with Strong Team, Good Weather, Favorable Pitch:
+------------+----------------+
| Outcome    |   phi(Outcome) |
+============+================+
| Outcome(0) |         0.2000 |
+------------+----------------+
| Outcome(1) |         0.8000 |
+------------+----------------+


In [ ]:
# Query 2: Given that the team is weak, weather is bad, and the pitch is unfavorable,
# what is the probability of winning?
result = inference.query(variables=['Outcome'], evidence={'Team Strength': 0, 'Weather': 0, 'Pitch Condition': 0})
print("\nPrediction with Weak Team, Bad Weather, Unfavorable Pitch:")
print(result)



Prediction with Weak Team, Bad Weather, Unfavorable Pitch:
+------------+----------------+
| Outcome    |   phi(Outcome) |
+============+================+
| Outcome(0) |         0.9000 |
+------------+----------------+
| Outcome(1) |         0.1000 |
+------------+----------------+


In [ ]:

# Query 3: Given that the team is strong and the weather is bad (ignoring pitch condition),
# what is the probability of winning?
result = inference.query(variables=['Outcome'], evidence={'Team Strength': 1, 'Weather': 0})
print("\nPrediction with Strong Team, Bad Weather (Pitch Unknown):")
print(result)


Prediction with Strong Team, Bad Weather (Pitch Unknown):
+------------+----------------+
| Outcome    |   phi(Outcome) |
+============+================+
| Outcome(0) |         0.2200 |
+------------+----------------+
| Outcome(1) |         0.7800 |
+------------+----------------+
